In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib.lines import Line2D
import os
import json
import matplotlib.ticker as ticker
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from scipy.interpolate import make_interp_spline, BSpline
os.chdir("C:/Users/Zber/Documents/Dev_program/OpenRadar")
from FER.utils import MapRecord
from sklearn.model_selection import train_test_split

In [3]:
def annotation_update(record_list, width=100, total_frame=300):
    for record in record_list:
        if record.num_frames < width:
            pad = (width - record.num_frames)//2
            if record.onset < pad:
                record.peak += pad*2

            elif (total_frame - record.peak) < pad:
                record.onset -= pad*2
            else:
                record.onset -= pad
                record.peak += pad
        else:
            pad = record.num_frames - width
            record.peak -= pad

        record.path = record.path.replace("Raw_0.bin","{}.npy").replace("\\", "/")

        if record.num_frames != 100:
            record.peak += 1
        assert record.num_frames == 100, 'the num of frames must equal to 100!'
    return record_list


def annotation_attention(record_list, width=30):
    for record in record_list:
        record.onset = math.floor(record.onset * 3 / 10)
        record.peak = record.onset + width - 1
        record.path = record.relative_path.replace("_{}.npy","")
    return record_list


def annotation_append():
    str_arr = []
    str_format = "{} {} {} {} {} {} {} {}"
    npy_path = "{}_{}"
    emotion = 'Neutral'
    subs = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5']

    for sub in subs:
        for i in range(0,30):
            path = (os.path.join(sub, npy_path.format(emotion, i,)) + '_{}.npy').replace("\\", "/")
            label = "0"
            onset = 51
            peak = 150
            offset = -1
            e1 = 0
            e2 = 0
            e3 = 0
            str_arr.append(str_format.format(path, label, onset, peak, offset, e1, e2 , e3))
    return str_arr


def data_split(record_list):
    labels = [r.label for r in record_list]
    train, test = train_test_split(record_list, test_size=0.2, random_state=25, stratify=labels)
    return train, test

def hm_2_frame(root_path, hm_path, frame_path):
    record_list = [MapRecord(x.strip().split(), root_path) for x in open(hm_path)]

    new_record_list = annotation_attention(record_list)
    str_format = "{} {} {} {}\n"
    with open(frame_path, 'w') as f:
        for record in new_record_list:
            f.write(str_format.format(record.path, record.onset, record.peak, record.label))
    print("Write {} Records to txt file".format(len(new_record_list)))

def hm_2_landmark(root_path, hm_path, frame_path):
    record_list = [MapRecord(x.strip().split(), root_path) for x in open(hm_path)]

    new_record_list = annotation_attention(record_list)
    str_format = "{} {} {} {}\n"
    with open(frame_path, 'w') as f:
        for record in new_record_list:
            re_pa = record.path.replace("_{}","")+".npy"
            f.write(str_format.format(re_pa, record.onset, record.peak, record.label))
    print("Write {} Records to txt file".format(len(new_record_list)))


### Heatmap annotation file to frame annotation file

In [4]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"
hm_train = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_new.txt"
frame_train = "C:\\Users\\Zber\\Desktop\\Subjects_Frames\\video_annotation_train_new.txt"
hm_2_frame(root_path, hm_train, frame_train)

hm_test = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_new.txt"
frame_test = "C:\\Users\\Zber\\Desktop\\Subjects_Frames\\video_annotation_test_new.txt"
hm_2_frame(root_path, hm_test, frame_test)

Write 973 Records to txt file
Write 242 Records to txt file


In [4]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"
hm_train = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_new.txt"
landmark_train = "C:\\Users\\Zber\\Desktop\\Subjects_Landmark\\landmark_annotation_train_new.txt"
hm_2_landmark(root_path, hm_train, landmark_train)

hm_test = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_new.txt"
landmark_test = "C:\\Users\\Zber\\Desktop\\Subjects_Landmark\\landmark_annotation_test_new.txt"
hm_2_landmark(root_path, hm_test, landmark_test)

Write 973 Records to txt file
Write 242 Records to txt file


In [5]:
og_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large\\heatmap_annotation_full_test.txt"
new_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large\\heatmap_annotation_full_test_new.txt"
og = open(og_path, 'r')
new = open(new_path, 'w')

def check_name(name):
    for exp in exp_list:
        if ("S5/"+exp) in name:
            return False
    return True

for l in og:
    if check_name(l):
        new.write(l)
og.close()
new.close()

NameError: name 'exp_list' is not defined

In [ ]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

ele = [-np.inf]

for record in record_list:
    
    npy_path_ele = record.path.format('ele')
    npy_path_azi = record.path.format('azi')

    ele = np.load(npy_path_ele).flatten()
    azi = np.load(npy_path_azi).flatten()

    ele = np.mean(ele)
    azi = np.mean(azi)

    if ele == np.NINF or azi == np.NINF:
        print(record.path)

C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Surprise_11_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Anger_7_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Anger_13_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Joy_4_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Fear_25_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Surprise_26_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Surprise_19_{}.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Neutral_29_{}.npy


In [ ]:
print(np.NINF)

-inf


# mean and std

In [ ]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

ele = [-np.inf]

for record in record_list:
    
    npy_path = record.path.format('ele')
    d = np.load(npy_path).flatten()
    mask = np.isin(d, ele)
    if len(d[mask]) >0:
        print(npy_path)
    arr.append(d)

arr = np.concatenate(arr, axis=0)

mean = np.mean(arr)
std = np.std(arr)
print("Mean : {}".format(mean))
print("STD : {}".format(std))


C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Surprise_11_ele.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Joy_4_ele.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Fear_25_ele.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Surprise_26_ele.npy
C:\Users\Zber\Desktop\Subjects_Heatmap\S5/Surprise_19_ele.npy
Mean : -inf
STD : nan


C:\Users\Zber\anaconda3\envs\Emo\lib\site-packages\numpy\core\_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [ ]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large\\heatmap_annotation_train.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]
train = []

ele = [-np.inf, np.inf, 0]

for record in record_list:
    
    npy_path = record.path.format('azi')
    d = np.load(npy_path).flatten()
    # mask = d > 1000
    mask = np.isin(d, ele)
    if len(d[mask]) >0:
        print(npy_path)
    # arr.append(d)
    else:
        train.append(record)

# arr = np.concatenate(arr, axis=0)


# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation_train.txt"), 'a') as f:
#     for record in train:
#         f.write(str_format.format(record.relative_path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(train)))

In [ ]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Frames"
# annotaton_path = "D:\\Subjects\\annotations_v2.txt"
# annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation.txt"
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
# annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test.txt"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

# new_record_list = annotation_update(record_list)
#
# # str format: path, label, onset, peak, offset, widthError, heightError, indexError
# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation.txt"), 'a') as f:
#     for record in new_record_list:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(new_record_list)))

# train, test = data_split(record_list)

# str format: path, label, onset, peak, offset, widthError, heightError, indexError
# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation_train.txt"), 'a') as f:
#     for record in train:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(train)))
#
# with open(os.path.join(root_path, "heatmap_annotation_test.txt"), 'a') as f:
#     for record in test:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(test)))


# str_arr = annotation_append()
# with open(annotaton_path, 'a') as f:
#     f.writelines('\n'.join(str_arr))
# print("Write {} Records to txt file".format(len(str_arr)))

new_record_list = annotation_attention(record_list)
str_format = "{} {} {} {}\n"
with open(os.path.join(root_path, "annotations_att_train.txt"), 'w') as f:
    for record in new_record_list:
        f.write(str_format.format(record.path, record.onset, record.peak, record.label))
print("Write {} Records to txt file".format(len(new_record_list)))

In [ ]:
annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
new_annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_full_train.txt"
root_path = ""
record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotation_path)]

str_format = "{} {} {} {} {} {} {} {}"
with open(new_annotation_path, 'w') as f:
    for record in record_list:
        record.onset = 0
        record.peak = 299
        f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
print("Write {} Records to txt file".format(len(record_list)))

Write 979 Records to txt file


In [ ]:
annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test.txt"
new_annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_full_test.txt"
root_path = ""
record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotation_path)]

str_format = "{} {} {} {} {} {} {} {}"
with open(new_annotation_path, 'w') as f:
    for record in record_list:
        record.onset = 0
        record.peak = 299
        f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
print("Write {} Records to txt file".format(len(record_list)))

Write 252 Records to txt file


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NST(nn.Module):
	def __init__(self):
		super(NST, self).__init__()

	def forward(self, fm_s, fm_t):
		fm_s = fm_s.view(fm_s.size(0), fm_s.size(1), -1)
		fm_s = F.normalize(fm_s, dim=2)

		fm_t = fm_t.view(fm_t.size(0), fm_t.size(1), -1)
		fm_t = F.normalize(fm_t, dim=2)

		loss = self.poly_kernel(fm_t, fm_t).mean() \
			 + self.poly_kernel(fm_s, fm_s).mean() \
			 - 2 * self.poly_kernel(fm_s, fm_t).mean()

		return loss

	def poly_kernel(self, fm1, fm2):
		fm1 = fm1.unsqueeze(1)
		fm2 = fm2.unsqueeze(2)
		out = (fm1 * fm2).sum(-1).pow(2)

		return out

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 4, requires_grad=True)
model = NST()

loss = model(input, target)

print(loss.value)


In [ ]:
class SP(nn.Module):
	'''
	Similarity-Preserving Knowledge Distillation
	https://arxiv.org/pdf/1907.09682.pdf
	'''
	def __init__(self):
		super(SP, self).__init__()

	def forward(self, fm_s, fm_t):
		fm_s = fm_s.view(fm_s.size(0), -1)
		G_s  = torch.mm(fm_s, fm_s.t())
		norm_G_s = F.normalize(G_s, p=2, dim=1)

		fm_t = fm_t.view(fm_t.size(0), -1)
		G_t  = torch.mm(fm_t, fm_t.t())
		norm_G_t = F.normalize(G_t, p=2, dim=1)

		loss = F.mse_loss(norm_G_s, norm_G_t)

		return loss

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 4, requires_grad=True)
model = SP()
loss = model(input, target)
print(loss.detach().numpy())

In [ ]:
os.chdir("../..")
from FER.em_network.models.c3d import C3DFusionBaseline
device = 'cuda'
m = C3DFusionBaseline(100, 7)
i = 0
for p in m.parameters():
    print(p.size())
    i += 1

print(i)

torch.Size([16, 1, 3, 7, 3])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([32, 16, 3, 7, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([64, 32, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([16, 1, 3, 7, 3])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([32, 16, 3, 7, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([64, 32, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 7, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([1024, 6144])
torch

In [ ]:
from FER
root = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large"
path = "S2/Disgust_0_{}"


In [ ]:
35 * 16

560